In [3]:
from keras.preprocessing.image import load_img, img_to_array

target_image_path = 'imgs/portrait.jpg'
style_reference_image_path = 'imgs/transfer_style_reference.jpg'

width, height = load_img(target_image_path).size

img_height = 400

img_width = int(width * img_height / height)


In [4]:
import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0 ,255).astype('uint8')
    return x

In [6]:
from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3))

input_tensor = K.concatenate([target_image, style_reference_image, combination_image], axis=0)

model = vgg19.VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)
print('Model load')

Model load


In [7]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [8]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0 ,1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S-C)) / (4. * (channels ** 2) * (size ** 2))

In [9]:
def total_variation_loss(x):
    a = K.square(x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, img_width - 1, :])
    b = K.square(x[:, :img_height - 1, :img_width - 1, :] - x[:, img_height - 1, 1, :])
    return K.sum(K.pow(a + b, 1.25))
    

In [15]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = 'block5_conv2'
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss.assign_add(content_weight * content_loss(target_image_features, combination_features))

for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss.assign_add((style_weight / len(style_layers)) * sl)
    
loss.assign_add(total_variation_weight * total_variation_loss(combination_image))

ValueError: Dimensions must be equal, but are 608 and 399 for '{{node Sub_9}} = Sub[T=DT_FLOAT](strided_slice_28, strided_slice_29)' with input shapes: [1,399,608,3], [1,399,3].